In [3]:
import os

os.chdir("..")

In [4]:
import pandas as pd

from country_abbrev import *
from country_language import *
from pytrends.request import TrendReq

import pycountry
import itertools

from googletrans import LANGCODES
import swifter

import trends_helpers
import numpy as np

In [5]:
currencies = pd.read_excel("countries-codes-and-currencies-(2020).xlsx")

In [6]:
keywords_list= list(set(currencies["Currency Name"]))
len(keywords_list)

141

In [5]:
countries_iso3 = currencies["Country Code"]
countries = [pycountry.countries.get(alpha_3=code).alpha_2 for code in countries_iso3]

In [6]:
# list of all unique languages:
unique_languages = pd.Series(list(set(list(itertools.chain(*country_language_dict.values())))), name='language')

# list of language codes from googletrans
langcodes = pd.DataFrame.from_dict(LANGCODES, orient='index', columns=['code'])
langcodes.index = langcodes.index.str.capitalize()

refugee_lang = unique_languages.to_frame().merge(langcodes, left_on='language', right_index=True, how='left')

refugee_lang.dropna(inplace=True)

refugee_lang_not_en = refugee_lang[refugee_lang['code'] != 'en']

In [7]:
translated_keyword_currency = refugee_lang_not_en['code'].swifter.apply(lambda x: trends_helpers.translate_keywords_list(lst = keywords_list, lang= x))

Pandas Apply:   0%|          | 0/83 [00:00<?, ?it/s]

In [8]:
refugee_lang_not_en

,language,code
2,Latvian,lv
4,Tajik,tg
6,Khmer,km
7,Luxembourgish,lb
10,Romanian,ro
...,...,...
181,Kyrgyz,ky
182,Lithuanian,lt
183,Cebuano,ceb
185,Thai,th


In [9]:
columns = list(refugee_lang_not_en['code'])

df = pd.concat([pd.DataFrame(sublist, columns=[col]) for sublist, col in zip(translated_keyword_currency, columns)], axis=1)

df['en']=keywords_list

df.head()

,lv,tg,km,lb,ro,ko,no,st,lo,be,...,sq,id,bn,bs,ky,lt,ceb,th,uk,en
0,botsvānas nokrišņi,боришоти ботсвана,ភ្លៀងធ្លាក់ botswana,botswana nidderschlag,botswana precipitații,보츠와나 강우량,botswana nedbør,botswana pula,ຝົນ botswana,батсвана ападкі,...,reshjet e botsvanës,curah hujan botswana,বতসোয়ানা বৃষ্টিপাত,padavine u bocvani,ботсвана жаан-чачын,botsvanos krituliai,pag-ulan sa botswana,บอตสวานาฝนตก,ботсвана опади,Botswana Pula
1,zālamana dolārs,доллари сулаймон,ដុល្លារសាឡូម៉ូន,solomon dollar,dolarul solomon,솔로몬 달러,solomon dollar,solomon dollar,ໂດລາໂຊໂລມອນ,саламонаў даляр,...,dollar solomon,dolar sulaiman,সলোমন ডলার,solomon dollar,соломон доллары,saliamono doleris,dolyar ni solomon,ดอลลาร์โซโลมอน,соломонов долар,Solomon Dollar
2,jamaikas dolārs,доллари ямайка,ដុល្លារហ្សាម៉ាអ៊ីក,jamaikaneschen dollar,dolarul jamaican,자메이카 달러,jamaicanske dollar,dollar ea jamaica,ໂດລາຈາໄມກາ,ямайскі даляр,...,dollar xhamajkan,dolar jamaika,জ্যামাইকান ডলার,jamaican dollar,ямайка доллары,jamaikos doleris,dolyar sa jamaica,ดอลลาร์จาเมกา,ямайський долар,Jamaican Dollar
3,rands,ранд,រ៉ែន,rand,randul,랜드,randen,ranta,ແຣນ,рэнд,...,randi,rand,র্যান্ড,rand,ранд,randas,ang rand,แรนด์,ренд,Rand
4,bulgārijas lev,булғористон лев,លីវប៊ុលហ្គារី,bulgaresch lev,lev bulgar,불가리아 레프,bulgarsk lev,lev ea bulgaria,bulgarian lev,балгарскі леў,...,lev bullgar,lev bulgaria,বুলগেরিয়ান লেভ,bugarski lev,болгар лев,bulgarijos levas,bulgarian nga lev,เลฟบัลแกเรีย,болгарський лев,Bulgarian Lev


In [10]:
country_language_dict
max_length = max(map(len, country_language_dict.values()))

data_padded = {key: arr + [np.nan] * (max_length - len(arr)) for key, arr in country_language_dict.items()}


In [11]:
langs = pd.DataFrame(data_padded)
langs = langs.T
langs = langs.reset_index()

In [12]:
max_length = max(map(len, country_language_dict.values()))

data_padded = {key: arr + [np.nan] * (max_length - len(arr)) for key, arr in country_language_dict.items()}

langs = pd.DataFrame(data_padded)
langs = langs.T
langs = langs.reset_index()
langs = langs.rename(columns={'index': 'Country', 0:'lang1', 1:'lang2', 2:'lang3'})

# Apply the function to the 'Country' column
langs['ISO2'] = langs['Country'].apply(trends_helpers.get_iso2_country_code)

langs = langs.drop(columns=["Country"])
langs_long = pd.melt(langs, id_vars=['ISO2'], var_name='numlang', value_name='lang')
langs_long = langs_long.dropna()

In [13]:
langs_long = pd.merge(langs_long, refugee_lang_not_en, left_on="lang", right_on="language")

In [14]:
count_files=trends_helpers.file_counter("trends_downloads/currency_partial results/original_lang")
print("{:.1f}% done.".format((count_files+8)/len(countries)*100))

45.9% done.


In [16]:
from pytrends.request import TrendReq
import time


# Set up the Google Trends API object
# pytrends = TrendReq()

results = pd.DataFrame() 

# Loop through regions and keywords

count_files=trends_helpers.file_counter("trends_downloads/currency_partial results/original_lang")


for i, country in enumerate(langs_long["ISO2"][count_files+8:], start=count_files+8): # added +x a posteriori - empty dfs
    
    languagecode= langs_long["code"][i]

    for keyword in df[languagecode]:    
        print("Searching: " + str(keyword) + " in " + str(country))
        time.sleep(4)
        
        pytrends = TrendReq(tz=360, timeout=(10, 25), retries=2, backoff_factor=0.5,
            requests_args={'headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}})
        
        # Build the payload with the selected region, keyword and date range
        pytrends.build_payload(kw_list=[keyword], timeframe='2005-01-01 2023-01-01', geo=str(country))
        # Get the interest over time data for the selected keyword
        interest_over_time_df = pytrends.interest_over_time()

        # Check if the DataFrame is not empty
        if not interest_over_time_df.empty:
            interest_over_time_df= interest_over_time_df.reset_index()
            interest_over_time_df['trends_index'] = interest_over_time_df.iloc[:, 1] 
            interest_over_time_df = interest_over_time_df.drop(interest_over_time_df.columns[1], axis=1)

            interest_over_time_df = interest_over_time_df.drop(columns=["isPartial"], axis=1)
            interest_over_time_df["keyword"]=keyword
            interest_over_time_df["region"] = country
            
            # Append the results to the main dataframe
            results = pd.concat([results, interest_over_time_df])
    
    if not results.empty:
        results_country = results[results["region"]==country]
        if not results_country.empty:
            translation = {'keyword': df[langs_long["code"][i]], 'keyword_en': df["en"]}
            translation = pd.DataFrame(translation)
            results_country = pd.merge(results_country, translation, left_on="keyword", right_on="keyword", how="left")
            file_path = 'trends_downloads/currency_partial results/original_lang/' + str(country) + '_' + languagecode + '.csv'
            results_country.to_csv(file_path, index=False)
        else:
            count_files = count_files+1 # because the loop interrups, should find a way to store this to memory
    else:
        count_files = count_files+1
    time.sleep(300) # Wait a few minutes until starting with next country 


Searching: pluviométrie au botswana in LA
Searching: dollar de salomon in LA
Searching: dollar jamaïcain in LA
Searching: le rand in LA
Searching: lev bulgare in LA
Searching: eau de cologne costaricienne in LA
Searching: dollar bélizéen in LA
Searching: dirham marocain in LA
Searching: rail in LA
Searching: somoni in LA
Searching: rial iranien in LA
Searching: lac albanais in LA
Searching: dinars bahreïnis in LA
Searching: personnes in LA
Searching: manat in LA
Searching: marque convertible in LA
Searching: franc congolais in LA
Searching: le dalasi gambien in LA
Searching: cedi ghanéen in LA
Searching: dinar irakien in LA
Searching: peso chilien in LA
Searching: dollars américain in LA
Searching: roupie sri lankaise in LA
Searching: dollar de brunei in LA
Searching: renminbi yuan in LA
Searching: dinar libyen in LA
Searching: drachme in LA
Searching: hryvnia in LA
Searching: en roupie indonésienne in LA
Searching: nouveau dobra in LA
Searching: franc rwandais in LA
Searching: bolivia

RetryError: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22kwacha%22%2C+%22time%22%3A+%222005-01-01+2023-01-01%22%2C+%22geo%22%3A+%22SC%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))

In [24]:
results

""
